# DEFORESTATION SCENARIO PIPELINE

In [2]:
import digitalhub as dh
PROJECT_NAME = "deforestation"
proj = dh.get_or_create_project(PROJECT_NAME) # source="git://github.com/scc-digitalhu

Register to the open data space copenicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [19]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

### Download data from Sentinel 2

In [20]:
string_dict_data = """{
 "satelliteParams":{
     "satelliteType": "Sentinel2"
 },
 "startDate": "2018-01-01",
 "endDate": "2019-12-31",
 "geometry": "POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))",
 "area_sampling": "true",
 "cloudCover": "[0,20]",
 "artifact_name": "data" 
 }"""

list_args =  ["main.py",string_dict_data]
function_s2 = proj.new_function("download_images_s2",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.10.1",command="python")

In [ ]:
run = function_s2.run(action="job",
        secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
        fs_group='8877',
        args=list_args,           
        volumes=[{
            "volume_type": "persistent_volume_claim",
            "name": "volume-deforestation",
            "mount_path": "/app/files",
            "spec": {
                "size": "300Gi"
            }}])

In [29]:
run.refresh().status.state

'RUNNING'

In [11]:
artifact = proj.get_artifact('data')

### Perform Data Elaboration on RS-Tool

#### Input data (Shape file)

In [2]:
artifact_name='bosco'
src_path='bosco'
artifact_bosco = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

#### Input data (Tiles)

In [4]:
artifact_name='data'
src_path='data'
artifact_data = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

In [ ]:
function_rs = proj.new_function("elaborate",kind="container", image="ghcr.io/tn-aixpa/rs-deforestation:2.7_b2", code_src="launch.sh")

In [8]:
#!pip uninstall digitalhub -y && pip install git+https://github.com/scc-digitalhub/digitalhub-sdk
!pip uninstall digitalhub digitalhub-runtime-container -y && pip install git+https://github.com/scc-digitalhub/digitalhub-sdk git+https://github.com/scc-digitalhub/digitalhub-sdk-runtime-container 

Found existing installation: digitalhub 0.11.0b4
Uninstalling digitalhub-0.11.0b4:
  Successfully uninstalled digitalhub-0.11.0b4
Found existing installation: digitalhub-runtime-container 0.11.0b1
Uninstalling digitalhub-runtime-container-0.11.0b1:
  Successfully uninstalled digitalhub-runtime-container-0.11.0b1
  Cloning https://github.com/scc-digitalhub/digitalhub-sdk to /tmp/pip-req-build-vqw45v6r
  Running command git clone --filter=blob:none --quiet https://github.com/scc-digitalhub/digitalhub-sdk /tmp/pip-req-build-vqw45v6r
  Resolved https://github.com/scc-digitalhub/digitalhub-sdk to commit 2454c8f947e2690670565088473474df7004421b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/scc-digitalhub/digitalhub-sdk-runtime-container to /tmp/pip-req-build-__2ja3za
  Running command git clone --filter=blob:none --quiet https://github.com/scc-digitalhub/digitalhub-sdk-runtim

In [ ]:
run_el = function_rs.run(action="job",
            fs_group='8877',
            resources={
                "cpu": {
                    "requests": "3",
                    
                },
                "mem":{
                    "requests": "16Gi"
                }
            },
            profile="1xa100",
            volumes=[{
            "volume_type": "persistent_volume_claim",            
            "name": "volume-deforestation", # this name has to be equal to the name of the volume created in krm
            "mount_path": "/app/files",
            "spec": {
                "size": "5Gi"
            }}],
            args=['/shared/launch.sh', 'bosco','data',"['2018','2019']",'deforestation_output']
         )

In [8]:
run_el.refresh().status.state

'RUNNING'

In [ ]:
%%writefile "src/deforestation_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(startYear, endYear, geometry, input1, input2, outputName):
    with pipeline_context() as pc:

        string_dict_data = "{ \"satelliteParams\":{\"satelliteType\": \"Sentinel2\" }, \"startDate\": \"" + startYear + "-01-01\", \"endDate\": \"" + endYear + "-12-31\", \"geometry\": \"" + geometry + ", \"area_sampling\": \"true\", \"cloudCover\": \"[0,20]\", \"artifact_name\": \"data_s2_deforestation\" }"
        list_args =  ["main.py",string_dict_data]

        s1 = pc.step(
            name="download",
            function="download_images_s2",
            action="job",
            secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
            fs_group='8877',
            args=list_args,
            volumes=[{"volume_type": "persistent_volume_claim","name": "volume-s2","mount_path": "/app/files","spec": {"claim_name": "volume-s2"}}]
            ),

        yearRange = "[" + startYear + ',' + endYear + "]"
        print(yearRange)             
        s2 = pc.step(
            action="job",
            fs_group='8877',
            resources={
                "cpu": {
                    "requests": "3",
                    
                },
                "mem":{
                    "requests": "16Gi"
                }
            },
            profile="1xa100",
            volumes=[{
            "volume_type": "persistent_volume_claim",            
            "name": "volume-deforestation", # this name has to be equal to the name of the volume created in krm
            "mount_path": "/app/files",
            "spec": {
                "size": "5Gi"
            }}],
            args=['/shared/launch.sh', input1,'data_s2_deforestation',yearRange,outputName]
            ).after(s1)        
     